In [ ]:
def merge_sort(array, use_merge_improvement=False,
               use_insertion_sort=False, use_role_swap=False):
    """Merge sort (with optimizations)"""
    tmp_array = list(array)
    
    merge_sort_aux(array, tmp_array, 0, len(array) - 1, use_merge_improvement,
                   use_insertion_sort, use_role_swap)
    return array


In [ ]:
def merge_sort_aux(array_from, array_to, lo, hi,
                   use_merge_improvement=False,
                   use_insertion_sort=False,
                   use_role_swap=False):
    if hi <= lo:
        return

    mid = lo + (hi - lo) // 2
    merge_sort_aux(array_from, array_to, lo, mid, use_merge_improvement, use_insertion_sort, use_role_swap)
    merge_sort_aux(array_from, array_to, mid + 1, hi, use_merge_improvement, use_insertion_sort, use_role_swap)

    merge(array_from, array_to, lo, mid, hi, use_merge_improvement, use_role_swap)


In [ ]:
def merge(array_from, array_to, lo, mid, hi,
          use_merge_improvement=False, use_role_swap=False):
    
    i = lo
    j = mid + 1
    for k in range(lo, hi + 1):  # k = lo,...,hi
        if j > hi or (i <= mid and array_from[i] <= array_from[j]):
            array_to[k] = array_from[i]
            i += 1
        else:
            array_to[k] = array_from[j]
            j += 1
    for k in range(lo, hi + 1):  # k = lo,...,hi
        array_from[k] = array_to[k]

In [ ]:
# Sort array in range lo to hi with insertion sort
def insertion_sort(array, lo, hi):
    for i in range(lo, hi+1):
        val = array[i]
        j = i
        while j > lo and array[j - 1] > val:
            array[j] = array[j - 1]
            j -= 1
        array[j] = val

A simple example that you can use to test the correctness of your implementation with different activated imprevements

In [ ]:
# change here, what configuration you test
use_merge_improvement = True
use_insertion_sort = True
use_role_swap = True

test = [6, 2, 5, 8, 4, 5, 2, 4, 12, 3, 2, 5]
merge_sort(test, use_merge_improvement, use_insertion_sort, use_role_swap)

Try in any case ``use_swap`` in combination with the other improvements.

# Question 3d

The following code contains the implementation for the experiments in part d. You can ignore the details and simply use the calls of the code (below).

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import random
import timeit


def sort_all_test_instances(test_instances, use_merge_improvement=False,
        use_insertion_sort=False, use_role_swap=False):
    """Auxiliary function for the experiments.

    Sorts all test_instances with merge_sort and the given parameters
    for the different improvements.
    """
    for array in test_instances:
        a = list(array) # copy to not change the instance for the next run
        merge_sort(a, use_merge_improvement, use_insertion_sort, use_role_swap)

        
def run_experiment(instance_creator):
    """Run a complete experiment.

    The test arrays are created with instance_creator.
    """

    # We first generate arrays of different sizes, namely from
    # min_size to max_size with step width step. For each size,
    # we create instances_per_size many random arrays, with the
    # given by instance_creator.
    min_size = 10000
    max_size = 20000
    step = 2000
    instances_per_size = 2

    test_instances = dict()
    for size in range(min_size, max_size + 1, step):
        test_instances[size] = []
        for num in range(instances_per_size):
            array = instance_creator(size)
            test_instances[size].append(array)

    results = dict()

    # Internal auxiliary function, that runs a configuration on all instances and
    # measures and collects the times.
    def collect_data(use_merge_improvement=False, use_insertion_sort=False,
                     use_role_swap=False):
        print("Collecting data for merge sort with parameters",
              "use_merge_improvement =", use_merge_improvement,
              "use_insertion_sort =", use_insertion_sort,
              "use_role_swap =", use_role_swap)

        times = []
        for n in range(min_size, max_size + 1, step):
            cmd = (f"sort_all_test_instances(test_instances[{n}]," +
                   f"use_merge_improvement={use_merge_improvement}," +
                   f"use_insertion_sort={use_insertion_sort},"+
                   f"use_role_swap={use_role_swap})")
            # We run the configuration three times and use the best time
            # (to stabilize the results). Since sort_all_test_instances already
            # has instances_per_size many iterations, we on top of that only run
            # 10 repetitions in timeit.
            t = timeit.repeat(lambda:
                              sort_all_test_instances(test_instances[n],
                                  use_merge_improvement, use_insertion_sort,
                                  use_role_swap),
                              repeat=3, number=10, globals=globals())
            times.append(min(t)/(10*instances_per_size))
            print("size", n, "took", min(t)/(10*instances_per_size),
                  "seconds on average")
        results[(use_merge_improvement, use_insertion_sort, use_role_swap)] = times


    collect_data(use_merge_improvement=True)
    collect_data(use_insertion_sort=True)
    collect_data(use_role_swap=True)
    collect_data(use_merge_improvement=True, use_insertion_sort=True,
                 use_role_swap=True)
    collect_data()


    xdata =  list(range(min_size, max_size + 1, step))
    plt.plot(xdata, results[(False, False, False)], marker="o", ls="-",
            label="without improvement")
    plt.plot(xdata, results[(True, False, False)], marker="o", ls="-",
            label="merge improvement")
    plt.plot(xdata, results[(False, True, False)], marker="o", ls="-",
            label="insertion sort")
    plt.plot(xdata, results[(False, False, True)], marker="o", ls="-",
            label="role swap")
    plt.plot(xdata, results[(True, True, True)], marker="o",
            label="all improvements")
    plt.legend(loc="upper left")
    plt.xlabel("input size")
    plt.ylabel("seconds")
    plt.show()


The following functions create random arrays of the given size with certain properties.

In [ ]:
# unique entries in random order
def random_unique(size):
    array = list(range(size))
    random.shuffle(array)
    return array

# random entries from 10 possible values
def few_different_values(size):
    return list(random.randrange(10) for i in range(size))

# We begin with a sorted array, chose size/10 times two
# random positions and swap their content:
def almost_sorted(size):
    array = list(range(size))
    for iteration in range(size//10):
        i = random.randrange(size)
        j = random.randrange(size)
        array[i], array[j] = array[j], array[i]
    return array


We conduct the first experiment with the random array that contains unique entries (this can take some time). You can only make meaningful experiments once you solved questions 3a-c.

In [ ]:
run_experiment(random_unique)

In the next experiment we consider arrays with many repeated entries.

In [ ]:
run_experiment(few_different_values)

The last experiment uses an input where many elements are already in the correct order.

In [ ]:
run_experiment(almost_sorted)